In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from lsst.daf.butlerUtils import ExposureIdInfo

import lsst.afw.display            as afwDisplay
import lsst.afw.image              as afwImage
import lsst.afw.geom               as afwGeom
import lsst.afw.table              as afwTable

import lsst.meas.algorithms        as measAlg

Load the high-level "tasks" that process the pixels

In [ ]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.pipe.tasks.calibrate         import CalibrateTask
from lsst.meas.algorithms.detection    import SourceDetectionTask
from lsst.meas.deblender               import SourceDeblendTask
from lsst.meas.base                    import SingleFrameMeasurementTask

The next import will fail until you've explicitly built and setup the Kron extension

    git clone git@github.com:lsst/meas_extensions_photometryKron
    setup -r . -j
    scons -Q opt=3 -j 4
    
and then restarted your jupyter notebook

In [ ]:
import lsst.meas.extensions.photometryKron

Setup the displays (by default an interface to ds9)

In [ ]:
disp  = afwDisplay.Display(1);  disp.callbacks  = disp._callbacks
disp2 = afwDisplay.Display(2);  disp2.callbacks = disp2._callbacks

## Create the tasks

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
algMetadata = dafBase.PropertyList()

config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask =         CharacterizeImageTask(None, config=config)

sourceDetectionTask =   SourceDetectionTask(schema=schema)

sourceDeblendTask =     SourceDeblendTask(schema=schema)

config = SingleFrameMeasurementTask.ConfigClass()
config.doApplyApCorr = 'yes'
config.slots.apFlux = 'base_CircularApertureFlux_12_0'
config.plugins.names.add("ext_photometryKron_KronFlux")
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config,
                                                   algMetadata=algMetadata)

It is a bug that I need to add this field to the metadata

In [ ]:
algMetadata.set("ext_photometryKron_KronFlux_nRadiusForFlux",
                config.plugins["ext_photometryKron_KronFlux"].nRadiusForFlux)

## Time to process some data

#### Read the input data

In [ ]:
fileName = "example1.fits"
exposureIdInfo = ExposureIdInfo(0, 5)

exposure = afwImage.ExposureF(fileName)

#### Create the output table

In [ ]:
tab = afwTable.SourceTable.make(schema)

#### Process the pixels

##### Characterise the exposure (e.g. estimate the PSF)

In [ ]:
result = charImageTask.characterize(exposure, exposureIdInfo)

psfCellSet = result.psfCellSet     # we'll look at this data structure later

##### Detect objects (`sources')

In [ ]:
result = sourceDetectionTask.run(tab, exposure)
sources = result.sources

##### Deblend overlapping objects

In [ ]:
sourceDeblendTask.run(exposure, sources, exposure.getPsf())

##### Measure the objects' properties

In [ ]:
sourceMeasurementTask.run(exposure, sources)

#### Write the results to a FITS file (if desired)

In [ ]:
sources.writeFits("outputTable.fits")
exposure.writeFits("example1-out.fits")

### OK, we've finished the image processing.

In [ ]:
import lsst.afw.display.utils as afwDisplayUtils
import lsst.afw.image.utils as afwImageUtils

Define a boolean array that tells us which objects are 'good'; in this case:
 - No saturated pixels near their centres
 - terminal objects that haven't been further deblended

In [ ]:
sources = sources.copy(True)
good = np.logical_and.reduce([sources.get('base_PixelFlags_flag_saturatedCenter') == 0,
                              sources.get("deblend_nChild") == 0,
                              ])

#### Look at how well the aperture and Kron photometry agrees with the PSF measurements

In [ ]:
with afwImageUtils.CalibNoThrow():
    apMag = exposure.getCalib().getMagnitude(sources.getApFlux())
    kronMag = exposure.getCalib().getMagnitude(sources["ext_photometryKron_KronFlux_flux"])
    psfMag = exposure.getCalib().getMagnitude(sources.getPsfFlux())

for mag, name in [
                    (apMag, "aperture"),
                    (kronMag, "Kron")
                 ]:
    plt.plot(apMag[good], (psfMag - mag)[good], 'o', alpha=0.5, label=name)

plt.legend(loc='best')
plt.axhline(0.0, ls=':', color='black')
plt.xlim(14, 24.5)
plt.ylim(-0.2, 0.8)

plt.xlabel("apMag")
plt.ylabel("psfMag - mag")

plt.show()

In [ ]:
if True:
    disp2.mtv(exposure)
else:
    disp2.erase()

In [ ]:
Kron_nRadiusForFlux = algMetadata.get("ext_photometryKron_KronFlux_nRadiusForFlux")

with disp2.Buffering():
    for s in sources[good]:
        shape = s.getShape().clone()
        rDet = shape.getDeterminantRadius()

        shape.scale(s["ext_photometryKron_KronFlux_radius"]/rDet)
        disp2.dot(shape, *s.getCentroid(), ctype=afwDisplay.RED)

        shape.scale(Kron_nRadiusForFlux)
        disp2.dot(shape, *s.getCentroid(), ctype=afwDisplay.CYAN)